# Session 1: Using Updated Tutoring Prompt

## Set up LLM through API

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

from IPython.display import Image, display

from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
llm = init_chat_model("openai:gpt-4o-mini")

In [ ]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [ ]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", 
                                             "content": user_input}]},
                             config):
        for value in event.values():
            print("\nAssistant:", value["messages"][-1].content)


tutor_prompt = open('mollick_tutor_prompt.txt').read()
graph.update_state(config, 
    {"messages": tutor_prompt }
)
stream_graph_updates('Who are you?')

while True:
    try:
        user_input = input("\nUser: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "Who are you?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

## Evaluation
### Subjective Experience
### Other observations and quality of tutoring